In [1]:
# import packages
import pandas as pd
import numpy as np
import geopandas as gpd

In [11]:
# load gas station coordinates
gas_coord = pd.read_csv("datafiles/wa_gas_stations.csv")
print(gas_coord.shape)
gas_coord.head()

(2211, 3)


,Longitude,Latitude,name
0,-122.329806,48.393760,24 Hour Express Fuel
1,-120.916225,47.191712,76
2,-122.357577,47.238704,76
3,-122.291324,48.435227,76
4,-120.014523,47.839608,76


In [17]:
# load nri data
df_nri = pd.read_csv("nri_w_coord.csv", index_col=0)
df_nri.head()

,OID_,NRI_ID,STATE,STATEABBRV,STATEFIPS,COUNTY,COUNTYTYPE,COUNTYFIPS,STCOFIPS,TRACT,...,WNTW_EALT,WNTW_EALS,WNTW_EALR,WNTW_RISKS,WNTW_RISKR,NRI_VER,GEOID,INTPTLAT,INTPTLONG,zipcode
0,30470,T53049950200,Washington,WA,53,Pacific,County,49,53049,950200,...,966.205692,15.736898,Relatively Low,11.035544,Relatively Moderate,November 2021,5.304995e+10,46.744007,-123.682527,98577.0
1,30601,T53049950500,Washington,WA,53,Pacific,County,49,53049,950500,...,163.437208,8.703212,Relatively Low,5.517714,Relatively Low,November 2021,NaN,NaN,NaN,NaN
2,30885,T53033032800,Washington,WA,53,King,County,33,53033,32800,...,602.933373,13.447835,Relatively Low,6.429770,Relatively Low,November 2021,5.303303e+10,47.653074,-121.431277,98251.0
3,30924,T53033031501,Washington,WA,53,King,County,33,53033,31501,...,51.584165,5.925633,Very Low,3.098243,Very Low,November 2021,5.303303e+10,47.320380,-121.879143,98051.0
4,30949,T53033032007,Washington,WA,53,King,County,33,53033,32007,...,23.052431,4.530382,Very Low,2.086950,Very Low,November 2021,5.303303e+10,47.391163,-122.073658,98042.0


In [9]:
# list(df_nri.columns)

In [6]:
# load census tract boundary
census_bnd = gpd.read_file("geojson/census_tract_WA_2021.geojson")
census_bnd

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,NAMELSADCO,STATE_NAME,LSAD,ALAND,AWATER,geometry
0,53,033,010102,1400000US53033010102,53033010102,101.02,Census Tract 101.02,WA,King County,Washington,CT,1623225,2170065,"POLYGON ((-122.29099 47.57247, -122.28314 47.5..."
1,53,053,062901,1400000US53053062901,53053062901,629.01,Census Tract 629.01,WA,Pierce County,Washington,CT,966774,0,"POLYGON ((-122.48338 47.21488, -122.48338 47.2..."
2,53,053,072305,1400000US53053072305,53053072305,723.05,Census Tract 723.05,WA,Pierce County,Washington,CT,4066566,0,"POLYGON ((-122.52636 47.22459, -122.52171 47.2..."
3,53,067,990100,1400000US53067990100,53067990100,9901,Census Tract 9901,WA,Thurston County,Washington,CT,0,25548183,"POLYGON ((-122.69843 47.10377, -122.69382 47.1..."
4,53,077,001504,1400000US53077001504,53077001504,15.04,Census Tract 15.04,WA,Yakima County,Washington,CT,575361,0,"POLYGON ((-120.50096 46.60201, -120.49633 46.6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1775,53,035,092701,1400000US53035092701,53035092701,927.01,Census Tract 927.01,WA,Kitsap County,Washington,CT,24488813,1038346,"POLYGON ((-122.59454 47.48485, -122.59459 47.4..."
1776,53,041,972000,1400000US53041972000,53041972000,9720,Census Tract 9720,WA,Lewis County,Washington,CT,1748453872,14106405,"POLYGON ((-122.10434 46.48005, -122.10285 46.4..."
1777,53,033,006701,1400000US53033006701,53033006701,67.01,Census Tract 67.01,WA,King County,Washington,CT,380256,105441,"POLYGON ((-122.34431 47.63743, -122.34193 47.6..."
1778,53,013,960200,1400000US53013960200,53013960200,9602,Census Tract 9602,WA,Columbia County,Washington,CT,2249624779,12561759,"POLYGON ((-118.24152 46.56042, -118.22801 46.5..."


In [10]:
# divide gas station by census tract boundary, and assign GeoID
# join NRI by census tract GEOID

points = gpd.GeoDataFrame(gas_coord, geometry=gpd.points_from_xy(gas_coord.Longitude, gas_coord.Latitude))
points.crs = {'init': 'epsg:4674'}
within_points = gpd.sjoin(points, census_bnd, op = 'within')
within_points

/Users/clarissarjtai/miniforge3/envs/pyproj_env/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/clarissarjtai/miniforge3/envs/pyproj_env/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3318: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/var/folders/_y/qzymmsdj5jj6nb7_yhb63h0r0000gn/T/ipykernel_9440/1314738116.py:6: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: 

,Longitude,Latitude,name,geometry,index_right,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,NAMELSADCO,STATE_NAME,LSAD,ALAND,AWATER
0,-122.329806,48.393760,24 Hour Express Fuel,POINT (-122.32981 48.39376),706,53,057,952600,1400000US53057952600,53057952600,9526,Census Tract 9526,WA,Skagit County,Washington,CT,16954407,1106782
67,-122.365342,48.430254,76,POINT (-122.36534 48.43025),706,53,057,952600,1400000US53057952600,53057952600,9526,Census Tract 9526,WA,Skagit County,Washington,CT,16954407,1106782
555,-122.362159,48.427548,Texaco,POINT (-122.36216 48.42755),706,53,057,952600,1400000US53057952600,53057952600,9526,Census Tract 9526,WA,Skagit County,Washington,CT,16954407,1106782
878,-122.365851,48.430067,NaN,POINT (-122.36585 48.43007),706,53,057,952600,1400000US53057952600,53057952600,9526,Census Tract 9526,WA,Skagit County,Washington,CT,16954407,1106782
932,-122.328600,48.398618,76,POINT (-122.32860 48.39862),706,53,057,952600,1400000US53057952600,53057952600,9526,Census Tract 9526,WA,Skagit County,Washington,CT,16954407,1106782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2163,-122.103328,47.339985,NaN,POINT (-122.10333 47.33999),1644,53,033,031601,1400000US53033031601,53033031601,316.01,Census Tract 316.01,WA,King County,Washington,CT,14669540,418506
2173,-119.100599,46.226988,NaN,POINT (-119.10060 46.22699),1131,53,021,020101,1400000US53021020101,53021020101,201.01,Census Tract 201.01,WA,Franklin County,Washington,CT,10829762,4053261
2174,-119.101165,46.228679,NaN,POINT (-119.10117 46.22868),1740,53,021,020201,1400000US53021020201,53021020201,202.01,Census Tract 202.01,WA,Franklin County,Washington,CT,655092,0
2196,-123.141611,47.404813,NaN,POINT (-123.14161 47.40481),1372,53,045,960201,1400000US53045960201,53045960201,9602.01,Census Tract 9602.01,WA,Mason County,Washington,CT,77619817,45357136


In [26]:
# output dataset
within_points.GEOID = within_points.GEOID.astype(float)
nri_output = within_points[['Longitude', 'Latitude','GEOID']].merge(df_nri, left_on='GEOID', right_on='GEOID', how='left')
print(nri_output.shape)
nri_output.iloc[:,:20].head()

(2208, 373)


,Longitude,Latitude,GEOID,OID_,NRI_ID,STATE,STATEABBRV,STATEFIPS,COUNTY,COUNTYTYPE,COUNTYFIPS,STCOFIPS,TRACT,TRACTFIPS,POPULATION,BUILDVALUE,AGRIVALUE,AREA,RISK_SCORE,RISK_RATNG
0,-122.329806,48.393760,5.305795e+10,38518.0,T53057952600,Washington,WA,53.0,Skagit,County,57.0,53057.0,952600.0,5.305795e+10,3821.0,440632000.0,4.995570e+06,6.549788,25.815695,Relatively High
1,-122.365342,48.430254,5.305795e+10,38518.0,T53057952600,Washington,WA,53.0,Skagit,County,57.0,53057.0,952600.0,5.305795e+10,3821.0,440632000.0,4.995570e+06,6.549788,25.815695,Relatively High
2,-122.362159,48.427548,5.305795e+10,38518.0,T53057952600,Washington,WA,53.0,Skagit,County,57.0,53057.0,952600.0,5.305795e+10,3821.0,440632000.0,4.995570e+06,6.549788,25.815695,Relatively High
3,-122.365851,48.430067,5.305795e+10,38518.0,T53057952600,Washington,WA,53.0,Skagit,County,57.0,53057.0,952600.0,5.305795e+10,3821.0,440632000.0,4.995570e+06,6.549788,25.815695,Relatively High
4,-122.328600,48.398618,5.305795e+10,38518.0,T53057952600,Washington,WA,53.0,Skagit,County,57.0,53057.0,952600.0,5.305795e+10,3821.0,440632000.0,4.995570e+06,6.549788,25.815695,Relatively High


In [27]:
print(nri_output.columns)

Index(['Longitude', 'Latitude', 'GEOID', 'OID_', 'NRI_ID', 'STATE',
       'STATEABBRV', 'STATEFIPS', 'COUNTY', 'COUNTYTYPE',
       ...
       'WNTW_EALA', 'WNTW_EALT', 'WNTW_EALS', 'WNTW_EALR', 'WNTW_RISKS',
       'WNTW_RISKR', 'NRI_VER', 'INTPTLAT', 'INTPTLONG', 'zipcode'],
      dtype='object', length=373)


In [29]:
output = nri_output[['Longitude', 'Latitude', 'GEOID', 'COUNTY','POPULATION','BUILDVALUE','AGRIVALUE', 'AREA', 'RISK_SCORE', 'RISK_RATNG','INTPTLAT', 'INTPTLONG', 'zipcode']].sort_index()
output.to_csv("gas_station_w_nri.csv")